# Trabalho Final - Modelos de IA e Machine Learning

### Integrantes:

###    1) Jurandir Ventura - RM336634
###    2) Marcelo Preto - RM336632
###    3) Fabio Rizzi - RM336656
###    4) Thiago Alexandre – RM336583

# Exercício 1.2

Utilizando o arquivo **1.2_acidentes-apriori.csv** que é um recorte da base de dados abertas dos acidentes de todas as rodovias federais, disponível de forma completa em: https://portal.prf.gov.br/dados-abertos-acidentes pela Polícia Rodoviária Federal. Procure analisar quais são as regras mais relevantes para análise de acidentes, tentem–se colocar no lugar de cientista de dados da polícia rodoviária, quais são as regras relevantes? Existem itemsets pouco relevantes que poderiam ser descartados na análise? Realize alguns filtros nos itemsets para facilitar a análise das regras geradas.

In [25]:
#Importando as bibliotecas. Se houver algum erro, use "pip install Libname"
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [4]:
#Leitura do dataset .csv

#Lembrar que a origem da dataset é uma variável.
#Neste caso é proveniente da pasta de datasets do Github: https://github.com/fiapIA/modelos_de_ia_e_ml/data

dataset = pd.read_csv('data/1.2_acidentes-apriori.csv', sep=';', engine='python')


#Visualização das 10 primeiras linhas do dataset
dataset.head(10)

,dia_semana,uf,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,condicao_metereologica,tracado_via
0,terca-feira,SP,Falta de Atencao Conducao,Colisao com objeto estatico,Com Vitimas Feridas,Plena Noite,Ceu Claro,Curva
1,terca-feira,PR,Falta de Atencao Conducao,Colisao traseira,Com Vitimas Feridas,Plena Noite,Nublado,Reta
2,terca-feira,SC,Animais na Pista,Colisao com objeto estatico,Com Vitimas Feridas,Plena Noite,Ceu Claro,Reta
3,terca-feira,CE,Ingestao de Substancias Psicoativas,Colisao com objeto estatico,Com Vitimas Feridas,Plena Noite,Nublado,Viaduto
4,terca-feira,MG,Falta de Atencao Conducao,Colisao transversal,Com Vitimas Feridas,Plena Noite,Nublado,Intersecao de vias
5,terca-feira,SC,Ingestao de Substancias Psicoativas,Saida de leito carrocavel,Com Vitimas Feridas,Plena Noite,Ceu Claro,Reta
6,terca-feira,MS,Animais na Pista,Atropelamento de Animal,Sem Vitimas,Plena Noite,Ceu Claro,Reta
7,terca-feira,GO,Desobediencia nas normas de transito pelo cond...,Colisao transversal,Com Vitimas Feridas,Plena Noite,Ceu Claro,Reta
8,terca-feira,GO,Desobedi�ncia nas normas de transito pelo pede...,Atropelamento de Pedestre,Com Vitimas Fatais,Plena Noite,Chuva,Reta
9,terca-feira,RS,Ingestao de Alcool,Colisao frontal,Com Vitimas Feridas,Plena Noite,Nublado,Curva


In [9]:
# Primeiro verificamos se tem dados nulos, para limpeza
dataset.isnull().sum()

dia_semana                0
uf                        0
causa_acidente            0
tipo_acidente             0
classificacao_acidente    0
fase_dia                  0
condicao_metereologica    0
tracado_via               0
dtype: int64

In [10]:
#Informações gerais do dataset

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20882 entries, 0 to 20881
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   dia_semana              20882 non-null  object
 1   uf                      20882 non-null  object
 2   causa_acidente          20882 non-null  object
 3   tipo_acidente           20882 non-null  object
 4   classificacao_acidente  20882 non-null  object
 5   fase_dia                20882 non-null  object
 6   condicao_metereologica  20882 non-null  object
 7   tracado_via             20882 non-null  object
dtypes: object(8)
memory usage: 1.3+ MB


In [11]:
# Vamos agrupar e contabilizar a quantidade de acidentes e variáveis por coluna

classe_acidentes = 0

for col in dataset.columns:
    print('Agrupamento por', col.upper())
    j = dataset[col].value_counts()
    classe_acidentes += len(j)
    print(j, '\n')


Agrupamento por DIA_SEMANA
sabado           3491
domingo          3384
sexta-feira      3123
segunda-feira    2815
terca-feira      2771
quinta-feira     2703
quarta-feira     2595
Name: dia_semana, dtype: int64 

Agrupamento por UF
MG    2717
SC    2636
PR    2411
RJ    1448
RS    1389
SP    1377
BA    1047
GO    1041
ES     871
PE     858
MT     659
CE     538
PB     516
RO     443
MS     431
PI     415
RN     413
MA     342
DF     300
PA     248
SE     213
AL     211
TO     161
RR      70
AC      60
AP      41
AM      26
Name: uf, dtype: int64 

Agrupamento por CAUSA_ACIDENTE
Falta de Atencao Conducao                                                           7583
Desobediencia nas normas de transito pelo condutor                                  2391
Velocidade Incompativel                                                             1984
Ingestao de Alcool                                                                  1597
Nao guardar distancia de seguranca                        

In [8]:
# Quantidade de variáveis para classificar os acidentes
classe_acidentes

98

In [12]:
# Quantidade de linhas e colunas do Dataset. Como todos os dados são categóricos, então precisamos fazer One-hot encoding
dataset.shape

(20882, 8)

In [16]:
## Aplicando get_dummies do pandas

dataset_onehot = pd.get_dummies(dataset, prefix_sep='=')

dataset_onehot.head(10)

,dia_semana=domingo,dia_semana=quarta-feira,dia_semana=quinta-feira,dia_semana=sabado,dia_semana=segunda-feira,dia_semana=sexta-feira,dia_semana=terca-feira,uf=AC,uf=AL,uf=AM,...,tracado_via=Curva,tracado_via=Desvio Temporario,tracado_via=Intersecao de vias,tracado_via=Nao Informado,tracado_via=Ponte,tracado_via=Reta,tracado_via=Retorno Regulamentado,tracado_via=Rotatoria,tracado_via=Tunel,tracado_via=Viaduto
0,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


### Agora temos um dataset onde cada condição é definida com 1 para ocorrência e 0 para não ocorrência das 98 variáveis para classificar os acidentes

In [18]:
# agora que o dataset está preparado, vamos usar APRIORI do pacote MLXTEND para verificar as regras

dataset_apriori = apriori(dataset_onehot, min_support=0.03, use_colnames= True)

# Aí verificamos o agrupamento do min_support (Support):

dataset_apriori.sort_values(by='support', ascending=False)


,support,itemsets
37,0.760895,(classificacao_acidente=Com Vitimas Feridas)
51,0.610430,(tracado_via=Reta)
42,0.573221,(fase_dia=Pleno dia)
43,0.539556,(condicao_metereologica=Ceu Claro)
232,0.464371,"(tracado_via=Reta, classificacao_acidente=Com ..."
...,...,...
251,0.030170,"(tracado_via=Intersecao de vias, fase_dia=Plen..."
351,0.030170,"(uf=RS, fase_dia=Pleno dia, classificacao_acid..."
443,0.030170,"(condicao_metereologica=Chuva, fase_dia=Plena ..."
526,0.030122,"(fase_dia=Pleno dia, causa_acidente=Nao guarda..."


In [21]:
# Agora usamos o Association Rules do mesmo pacote MLXTEND com a métrica = confiança

def get_associations(df, min_confidence):
    a = association_rules(df, metric='confidence',
                          min_threshold=min_confidence)
    return a.iloc[:, [0,1,4,5]].sort_values(by='confidence',
                                            ascending=False)

In [37]:
#Pegamos a confiança entre 55 e 95% com intervalo de 5

for i in np.linspace(0.55, 0.95, 5):
    association = get_associations(dataset_apriori, i)
    percent = "{:.1f}".format(i * 100)
    print('Regras de Associação com', percent, '% de confiança =>', association.shape[0])

Regras de Associação com 55.0 % de confiança => 656
Regras de Associação com 65.0 % de confiança => 324
Regras de Associação com 75.0 % de confiança => 188
Regras de Associação com 85.0 % de confiança => 21
Regras de Associação com 95.0 % de confiança => 7


## Com o percentual de confiança e quantidade das regras de associação, vamos limitar a análise para confiança maior que 75%, que já possui um volume grande e com maior confiança, descartando os demais

In [44]:
# Aplicamos a função para a variável

regra_associacao = get_associations(dataset_apriori, 0.75)

# Assim temos um dataset no formato de antecendente, consequente, suporte e confiança. Vamos agrupar.
regra_associacao.groupby('consequents').count()

,antecedents,support,confidence
consequents,,,
(fase_dia=Pleno dia),5,5,5
(classificacao_acidente=Com Vitimas Feridas),173,173,173
"(fase_dia=Pleno dia, classificacao_acidente=Com Vitimas Feridas)",2,2,2
(tracado_via=Reta),8,8,8


In [49]:
# Com o agrupamento acima, vamos agora ordenar pelo support para verificar os mais relevantes de "Vitimas Feridas"

regras_finais = regra_associacao.sort_values(by = 'support', ascending=False)

In [51]:
pd.set_option('max_colwidth', 300)
regras_finais.head(15)

,antecedents,consequents,support,confidence
28,(tracado_via=Reta),(classificacao_acidente=Com Vitimas Feridas),0.464371,0.760728
23,(fase_dia=Pleno dia),(classificacao_acidente=Com Vitimas Feridas),0.454171,0.792314
24,(condicao_metereologica=Ceu Claro),(classificacao_acidente=Com Vitimas Feridas),0.409300,0.758587
12,(causa_acidente=Falta de Atencao Conducao),(classificacao_acidente=Com Vitimas Feridas),0.295709,0.814322
114,"(fase_dia=Pleno dia, tracado_via=Reta)",(classificacao_acidente=Com Vitimas Feridas),0.270616,0.795467
115,"(condicao_metereologica=Ceu Claro, tracado_via=Reta)",(classificacao_acidente=Com Vitimas Feridas),0.264486,0.757198
106,"(fase_dia=Pleno dia, condicao_metereologica=Ceu Claro)",(classificacao_acidente=Com Vitimas Feridas),0.241931,0.787406
84,"(causa_acidente=Falta de Atencao Conducao, tracado_via=Reta)",(classificacao_acidente=Com Vitimas Feridas),0.193181,0.817427
80,"(causa_acidente=Falta de Atencao Conducao, fase_dia=Pleno dia)",(classificacao_acidente=Com Vitimas Feridas),0.186189,0.827762
81,"(causa_acidente=Falta de Atencao Conducao, condicao_metereologica=Ceu Claro)",(classificacao_acidente=Com Vitimas Feridas),0.168183,0.816364


# CONCLUSÃO

## Após ordenar pelo Support e considerando o mais relevante para classificação de acidente com vítimas feridas (maior quantidade), temos evidências de que Via Reta e até o Céu Claro podem estar relacionados a maior velocidade dos carros e por consequência acidentes com vítimas. O Falta de Atenção é uma das regras mais relevantes.

## Foco em campanha de conscientização durante o dia seria importante para tentar reduzir tais números, com foco nos estados da Região Sul e Sudeste, pela quantidade bem maior de acidentes, comparados aos outros estados do país.